In [8]:
import re

In [1]:
#To build a reverse complimentary strand
def rc(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A','N':'N'}
    return("".join(complement.get(base, base) for base in reversed(seq)))


### Write into FASTA file sequences of identified orthologs

In [27]:
## Collect contigs names and coordinates, add an offset - extra sequence to both borders
offset = 200
d = {}
with open('./Key_Files/ManualCuration') as file1:
    for line in file1:
        if not line.startswith('#'):
            dat = line.strip().split('\t')
            d[dat[0]] = {}
            if dat[0] in ['GCA_003118495.1_macFus_1.0','GCA_002335545.1_Aspe_assembly01']:
                start = int(dat[2])
                stop = int(dat[3])
                d[dat[0]]['neat'] = [dat[1],start + offset,stop,'minus']

            else:          
                if dat[4] != 'Nan':
                    start = int(dat[2])
                    stop = int(dat[3])
                    if start > stop:
                        d[dat[0]]['neat'] = [dat[1],start + offset,stop - offset,'minus']
                    else:
                        d[dat[0]]['neat'] = [dat[1],start - offset,stop + offset,'plus']
            if dat[8] != 'Nan':
                start = int(dat[6])
                stop = int(dat[7])
                if start > stop:
                    d[dat[0]]['malat'] = [dat[5],start + offset,stop - offset,'minus']
                else:
                    d[dat[0]]['malat'] = [dat[5],start - offset,stop + offset,'plus']        
                    


In [7]:
#Gather reference sequences
ref = {}
ref_names = ['NEAT1_Mouse','NEAT1_Marsupials','NEAT1_Human','MALAT1_Mouse','MALAT1_Human']
for ref_name in ref_names:
    seq_file = ref_name + '.fasta'
    with open(seq_file) as file1:
        fl = 0
        for line in file1:
            if line.startswith('>') and fl == 0:
                l = []
                name = line.rstrip().lstrip('>')
                fl = 1
            elif line.startswith('>') and fl == 1:
                ref[name] = ''.join(l)
                l = []
                name = line.rstrip().lstrip('>')
            elif not line.startswith('>'):
                l.append(line.strip())
        ref[name] = ''.join(l)
print(ref.keys())

dict_keys(['Mouse_NEAT1', 'Marsupials_NEAT1', 'Human_NEAT1', 'Mouse_MALAT1', 'Human_MALAT1'])


In [22]:
# Create a single smaller file with contigs on which orthologs were identified

out = open('NeededContigsAllGenomes.fasta','w')
for item in d:
    seq_file = './downloads/' + item + '_genomic.fna'
    acc = []
    if 'neat' in d[item]:
        acc.append(d[item]['neat'][0])
    else:
        acc.append('Nan')
    if 'malat' in d[item]:
        acc.append(d[item]['malat'][0])
    else:
        acc.append('Nan')
    with open(seq_file) as file1:
        fl = 0
        for line in file1:
            if line.startswith('>') and fl == 0:
                l = []
                name = line.lstrip('>').split(' ')[0]
                fl = 1
            elif line.startswith('>') and fl == 1:
                if name in acc:
                    out.write('>' + item + '|' + name + '\n' + ''.join(l).upper() + '\n')
                    
                l = []
                name = line.lstrip('>').split(' ')[0]
            elif not line.startswith('>'):
                l.append(line.strip())
        if name in acc:
            out.write('>' + item + '|' + name + '\n' + ''.join(l).upper() + '\n')
out.close()


In [28]:
out_neat = open('neat1_seqs_unrefined_offset_200.fasta','w')
out_malat = open('malat1_seqs_unrefined_offset_200.fasta','w')

out_malat.write('>Mouse_MALAT1\n' + ref['Mouse_MALAT1'] + '\n')
out_malat.write('>Human_MALAT1\n' + ref['Human_MALAT1'] + '\n')

for item in ['Mouse_NEAT1', 'Marsupials_NEAT1', 'Human_NEAT1']:
    out_neat.write('>' + item + '\n' + ref[item] + '\n')

# function to properly cut the gene depending on the strand
def processing(lib, accession, seq):
    global d
    # Cut out NEAT1 gene
    if 'neat' in d[lib] and accession == d[lib]['neat'][0]:
        if d[lib]['neat'][3] == 'plus':
            if (d[lib]['neat'][2] + 1) <= len(seq):
                cutout = seq[d[lib]['neat'][1]:d[lib]['neat'][2] + 1]
            else:
                cutout = seq[d[lib]['neat'][1]:]
            cutoutN = re.sub('N+','NN',cutout)
            out_neat.write('>' + lib + '_NEAT1\n' + cutoutN + '\n')
        elif d[lib]['neat'][3] == 'minus':
            if (d[lib]['neat'][1] + 1) <= len(seq):
                cutout = seq[d[lib]['neat'][2]:d[lib]['neat'][1] + 1]
            else:
                cutout = seq[d[lib]['neat'][2]:]
            cutoutN = re.sub('N+','NN',cutout)
            out_neat.write('>' + lib + '_NEAT1\n' + rc(cutoutN) + '\n')
        else:
            print('NEAT1',lib)
    # Cut out MALAT1 gene
    if 'malat' in d[lib] and accession == d[lib]['malat'][0]:
        if d[lib]['malat'][3] == 'plus':
            if (d[lib]['malat'][2] + 1) < len(seq):
                cutout = seq[d[lib]['malat'][1]:d[lib]['malat'][2] + 1]
            else:
                cutout = seq[d[lib]['malat'][1]:]
            cutoutN = re.sub('N+','NN',cutout)
            out_malat.write('>' + lib + '_MALAT1\n' + cutoutN + '\n')
        elif d[lib]['malat'][3] == 'minus':
            if (d[lib]['malat'][1] + 1) < len(seq):
                cutout = seq[d[lib]['malat'][2]:d[lib]['malat'][1] + 1]
            else:
                cutout = seq[d[lib]['malat'][2]:]
            cutoutN = re.sub('N+','NN',cutout)
            out_malat.write('>' + lib + '_MALAT1\n' + rc(cutoutN) + '\n')
        else:
            print('MALAT1',lib)
 
        
#Parsing the FASTA file with contigs and cut the orthologs

with open('NeededContigsAllGenomes.fasta') as file1:
    fl = 0
    for line in file1:
        if line.startswith('>') and fl == 0:
            l = []
            name = line.lstrip('>').split('|')[0]
            accession = line.strip().split('|')[1]
            fl = 1
        elif line.startswith('>') and fl == 1:
            processing(name, accession, ''.join(l).upper())
            l = []
            name = line.lstrip('>').split('|')[0]
            accession = line.strip().split('|')[1]
        elif not line.startswith('>'):
            l.append(line.strip())
    processing(name, accession, ''.join(l).upper())
out_neat.close()
out_malat.close()

### Multiple Sequence Alignment and preparation for it

To identify more accurately gene coordinates and structural features we performed MSA of NEAT1 orthologs in groups, which would be big enough and close phylogenetically as we expect high divergence of the primary sequence

In [43]:
#Defining the groups
groups = {'MarsMono':['MONOTREMATA','DIDELPHIMORPHIA','MICROBIOTHERIA','DIPROTODONTIA','DASYUROMORPHIA'],
'EuPerisPholi': ['EULIPOTYPHLA','PERISSODACTYLA','PHOLIDOTA'],          
'AfroSirTub':['MACROSCELIDEA','PILOSA','PROBOSCIDEA','AFROSORICIDA','CINGULATA','SIRENIA','TUBULIDENTATA','HYRACOIDEA'],
'RodLagoScan':['LAGOMORPHA','RODENTIA','SCANDENTIA'],
'PrimDerm':['PRIMATES','DERMOPTERA'],
'Art':['ARTIODACTYLA'],'Chirop':['CHIROPTERA'],'Carn':['CARNIVORA']}
groups_r = {}
for group in groups:
    for item in groups[group]:
        groups_r[item] = group


In [31]:
#Retrieve taxonomy class information for the genomes
tax = {}
with open('./Key_Files/Metadata') as file1:
    for line in file1:
        if not line.startswith('#'):
            dat = line.strip().split('\t')
            tax[dat[0]] = dat[7].upper()
       

In [52]:
#Write sequences in separate FASTA files together with the most relevant known ortholog
order = [group for group in groups]
handler = [open('./MSA/RAW/NEAT1_' + group + '.fasta','w') for group in order]

for i in range(len(order)):
    if order[i] == 'MarsMono':
        handler[i].write('>Marsupials_NEAT1\n' + ref['Marsupials_NEAT1'] + '\n')
    elif order[i] in ['PrimDerm','Art','Carn','EuPerisPholi']:
        handler[i].write('>Human_NEAT1\n' + ref['Human_NEAT1'] + '\n')
    elif order[i] == 'RodLagoScan':
        handler[i].write('>Mouse_NEAT1\n' + ref['Mouse_NEAT1'] + '\n')
    else:
        for item in ['Mouse_NEAT1', 'Human_NEAT1']:
            handler[i].write('>' + item + '\n' + ref[item] + '\n')


with open('neat1_seqs_unrefined_offset_200.fasta') as file1:
    fl = 0
    for line in file1:
        if line.startswith('>') and fl == 0:
            l = []
            name = '_'.join(line.lstrip('>').split('_')[:-1])

            fl = 1
        elif line.startswith('>') and fl == 1:
            if name in tax:
                idx = order.index(groups_r[tax[name]])
                handler[idx].write('>' + name + '\n' + ''.join(l) + '\n')
            else:
                print(name)
            l = []
            name = '_'.join(line.lstrip('>').split('_')[:-1])
        elif not line.startswith('>'):
            l.append(line.strip())
    if name in tax:
        idx = order.index(groups_r[tax[name]])
        handler[idx].write('>' + name + '\n' + ''.join(l) + '\n')

[i.close() for i in handler]

handler = [open('./MSA/RAW/MALAT1_' + group + '.fasta','w') for group in order]
for item in handler:
    item.write('>Mouse_MALAT1\n' + ref['Mouse_MALAT1'] + '\n')
    item.write('>Human_MALAT1\n' + ref['Human_MALAT1'] + '\n')
with open('malat1_seqs_unrefined_offset_200.fasta') as file1:
    fl = 0
    for line in file1:
        if line.startswith('>') and fl == 0:
            l = []
            name = '_'.join(line.lstrip('>').split('_')[:-1])

            fl = 1
        elif line.startswith('>') and fl == 1:
            if name in tax:
                idx = order.index(groups_r[tax[name]])
                handler[idx].write('>' + name + '\n' + ''.join(l) + '\n')
            else:
                print(name)
            l = []
            name = '_'.join(line.lstrip('>').split('_')[:-1])
        elif not line.startswith('>'):
            l.append(line.strip())
    if name in tax:
        idx = order.index(groups_r[tax[name]])
        handler[idx].write('>' + name + '\n' + ''.join(l) + '\n')

[i.close() for i in handler]

Mouse
Marsupials
Human
Mouse
Human


[None, None, None, None, None, None, None, None]

In [53]:
#In command line MSA was performed

#for i in `ls *.fasta`; do mafft "$i" > ./MSA/msa_"$i"; done

MSAs (made by MAFFT) of approximately cut sequences were visually explored in AlignmentViewer. 'TATA'-boxex together with 'triple helix' patterns were identified. Coordinates of these features were manually checked and put in the dictionary below. Precise gene sequences and structural elements were retrieved. 

### Retrieval of final sequences of orthologs and their gene features

In [1]:
coor = {'NEAT1':{'MarsMono':{'gene':[486,63741], 'trna':[63741,63805],'TATA':[438,486],'promoter':[240,443],'triplex':[63643,63741]},
'EuPerisPholi': {'gene':[346,40815], 'trna':[40815,40880],'TATA':[303,346],'promoter':[210,308],'triplex':[40727,40815]},          
'AfroSirTub':{'gene':[327,42714], 'trna':[42714,42780],'TATA':[289,327],'promoter':[80,294],'triplex':[42623,42714]},
'RodLagoScan':{'gene':[490,120889], 'trna':[120889,120965],'TATA':[448,490],'promoter':[280,453],'triplex':[120791,120889]},
'PrimDerm':{'gene':[349,68528], 'trna':[68528,68590],'TATA':[313,349],'promoter':[150,318],'triplex':[68439,68528]},
'Art':{'gene':[290,67634], 'trna':[67634,67700],'TATA':[255,290],'promoter':[40,259],'triplex':[67544,67634]},
'Chirop':{'gene':[343,40898], 'trna':[40898,40960],'TATA':[301,343],'promoter':[140,306],'triplex':[40798,40898]},
'Carn':{'gene':[301,73252], 'trna':[73253,73338],'TATA':[265,301],'promoter':[40,270],'triplex':[73064,73253]}},
        
       'MALAT1':{'MarsMono':{'gene':[2748,14813], 'trna':[14813,14880],'TATA':[2714,2748],'promoter':[2600,2719],'triplex':[14726,14813]},
'EuPerisPholi': {'gene':[2175,13550], 'trna':[13550,13610],'TATA':[2078,2175],'promoter':[1700,2083],'triplex':[13461,13550]},          
'AfroSirTub':{'gene':[1950,10572], 'trna':[10572,10640],'TATA':[1916,1950],'promoter':[1710,1921],'triplex':[10482,10572]},
'RodLagoScan':{'gene':[2793,15642], 'trna':[15642,15707],'TATA':[2723,2793],'promoter':[2590,2728],'triplex':[15548,15642]},
'PrimDerm':{'gene':[1979,13170], 'trna':[13170,13240],'TATA':[1945,1979],'promoter':[1730,1950],'triplex':[13073,13170]},
'Art':{'gene':[1332,21534], 'trna':[21534,21600],'TATA':[1298,1332],'promoter':[1150,1303],'triplex':[20318,21534]},
'Chirop':{'gene':[1327,11403], 'trna':[11403,11470],'TATA':[1293,1327],'promoter':[1100,1298],'triplex':[11310,11403]},
'Carn':{'gene':[1695,10552], 'trna':[10552,10615],'TATA':[1657,1695],'promoter':[1540,1662],'triplex':[10460,10552]}}}

In [3]:
# Write in FASTA files final versions of orthologs and their gene features
for gene in coor:
    for group in coor[gene]:
        msa = {}
        with open('./MSA/RAW/MSA/msa_' + gene + '_' + group + '.fasta') as file1:
            fl = 0
            for line in file1:
                if line.startswith('>') and fl == 0:
                    l = []
                    name = line.rstrip().lstrip('>')
                    fl = 1
                elif line.startswith('>') and fl == 1:
                    msa[name] = ''.join(l)
                    l = []
                    name = line.rstrip().lstrip('>')
                elif not line.startswith('>'):
                    l.append(line.strip())
            msa[name] = ''.join(l)
        for feature in coor[gene][group]:
            out = open('./FinalSeqs/' + gene + '_' + feature + '_' + group + '.fasta','w')
            start = coor[gene][group][feature][0]
            stop = coor[gene][group][feature][1]
            for item in msa:
                seq = ''.join(list(msa[item])[start:stop + 1]).replace('-','').upper()
                if seq and len(seq) > 20:
                    out.write('>' + item + '\n' + seq + '\n')
            
            out.close()

In [2]:
#Combine groups into a single file for each of features
features = ['gene','trna','TATA','promoter','triplex']
for gene in coor:
    for feature in features:
        out = open('./FinalSeqs/' + gene + '_' + feature + '_all.fasta','w')
        msa = {}
        for group in coor[gene]:
            with open('./FinalSeqs/' + gene + '_' + feature + '_' + group + '.fasta') as file1:
                fl = 0
                for line in file1:
                    if line.startswith('>'):
                        name = line
                    else:
                        msa[name] = line
        for item in msa:
            out.write(item + msa[item])
        out.close()